In [1]:
import sys; sys.path.append("../modules")
import random

import numpy as np
import sympy; sympy.init_printing()

import Permutations as pm
from Grid import *

from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

#from gridlink import * 
import gridlink as gl